In [3]:
!pip install -r requirements.txt --quiet


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [71]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file


True

In [11]:
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "int-vec")
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]

In [12]:
# search blob datasource connection string is optional - defaults to blob connection string
# This field is only necessary if you are using MI to connect to the data source
# https://learn.microsoft.com/azure/search/search-howto-indexing-azure-blob-storage#supported-credentials-and-connection-strings
search_blob_connection_string = os.getenv("SEARCH_BLOB_DATASOURCE_CONNECTION_STRING", blob_connection_string)
blob_container_name = os.getenv("BLOB_CONTAINER_NAME", "int-vec")
azure_openai_endpoint = os.environ["ENDPOINT_URL"]
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")

azure_openai_embedding_deployment = os.getenv("EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002")
azure_openai_model_name = os.getenv("EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002")
azure_openai_model_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1024))
# This field is only necessary if you want to use OCR to scan PDFs in the data source
azure_ai_services_key = os.getenv("AZURE_AI_SERVICES_KEY", "")

use_ocr = len(azure_ai_services_key) > 0
# OCR must be used to add page numbers
add_page_numbers = use_ocr

In [14]:
from azure.storage.blob import BlobServiceClient  
import glob

def upload_sample_documents(
        blob_connection_string: str,
        blob_container_name: str,
        documents_directory: str,
        # Set to false if you want to use credentials included in the blob connection string
        # Otherwise your identity will be used as credentials
        use_user_identity: bool = True,
    ):
        # Connect to Blob Storage
        blob_service_client = BlobServiceClient.from_connection_string(logging_enable=True, conn_str=blob_connection_string, credential=DefaultAzureCredential() if use_user_identity else None)
        container_client = blob_service_client.get_container_client(blob_container_name)
        if not container_client.exists():
            container_client.create_container()

        pdf_files = glob.glob(os.path.join(documents_directory, '*.pdf'))
        for file in pdf_files:
            with open(file, "rb") as data:
                name = os.path.basename(file)
                if not container_client.get_blob_client(name).exists():
                    container_client.upload_blob(name=name, data=data)

def upload_documents_without_ocr():
    upload_sample_documents(
        blob_connection_string=blob_connection_string,
        blob_container_name=blob_container_name,
        documents_directory=os.path.join("..", "..", "data", "documents")
    )

def upload_documents_with_ocr():
    upload_sample_documents(
        blob_connection_string=blob_connection_string,
        blob_container_name=blob_container_name,
        documents_directory = os.path.join("..", "..", "data", "ocrdocuments")
    )

if use_ocr:
     upload_documents_with_ocr()
else:
     upload_documents_without_ocr()

print(f"Setup sample data in {blob_container_name}")

Setup sample data in int-vec


In [ ]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)
from azure.search.documents.indexes.models import NativeBlobSoftDeleteDeletionDetectionPolicy

# Create a data source 
indexer_client = SearchIndexerClient(endpoint, credential)
container = SearchIndexerDataContainer(name=blob_container_name)
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type="azureblob",
    connection_string=search_blob_connection_string,
    container=container,
    data_deletion_detection_policy=NativeBlobSoftDeleteDeletionDetectionPolicy()
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

ResourceNotFoundError: (404) Resource not found
Code: 404
Message: Resource not found

In [58]:
import fitz  # PyMuPDF
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
 
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text_data = []
 
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        text_data.append({"page_no": str(page_num+1), "text": text})
 
    return text_data

In [59]:
text_data = extract_text_from_pdf('proposals/file_27.pdf')

In [68]:
text_data = extract_text_from_pdf('proposals/file_27.pdf')
text_data

[{'page_no': '1',
  'text': 'TEAM 70:\nDATASET 5: \nShelf Behaviors CCTV Data  \n'},
 {'page_no': '2',
  'text': 'TEAM\nMEMBERS\nLe Thuong\nMember\nTrung Dung\nTeam Leader\nThe Vinh\nMember\nThanh Hao\nMember\nHai Yen\nMember\n'},
 {'page_no': '3',
  'text': 'Agenda\nIntroduction \nProblem Statement\nSolution Overview\nMethodologies\nCore Functionality\nPerformance Metrics\nTimeline and Roadmap\nUser Interface (UI) or Interaction\nLimitations and Future Enhancements\nConclusion\n1.\n2.\n3.\n4.\n5.\n6.\n7.\n8.\n9.\n10.\n'},
 {'page_no': '4',
  'text': 'Proposal: Develop an AI solution utilizing video surveillance data\n for in-depth analysis of customer behavior.\n1.INTRO:\nIntroduction to the importance of understanding \ncustomer behavior in retail environments.\n'},
 {'page_no': '5',
  'text': 'Challenges and inefficiencies faced by retail businesses.\nSuboptimal store layout due to a lack of understanding of customer navigation.\nIneffective marketing strategies without detailed cus

In [69]:
formatted_text = "\n".join([page["text"] for page in text_data])
print(formatted_text)

TEAM 70:
DATASET 5: 
Shelf Behaviors CCTV Data  

TEAM
MEMBERS
Le Thuong
Member
Trung Dung
Team Leader
The Vinh
Member
Thanh Hao
Member
Hai Yen
Member

Agenda
Introduction 
Problem Statement
Solution Overview
Methodologies
Core Functionality
Performance Metrics
Timeline and Roadmap
User Interface (UI) or Interaction
Limitations and Future Enhancements
Conclusion
1.
2.
3.
4.
5.
6.
7.
8.
9.
10.

Proposal: Develop an AI solution utilizing video surveillance data
 for in-depth analysis of customer behavior.
1.INTRO:
Introduction to the importance of understanding 
customer behavior in retail environments.

Challenges and inefficiencies faced by retail businesses.
Suboptimal store layout due to a lack of understanding of customer navigation.
Ineffective marketing strategies without detailed customer insights.
Decreased customer satisfaction due to a lack of understanding of needs and
preferences.
2.  PROBLEM STATEMENT
DEFINING THE PROBLEM
Core issue addressed by the MVP: Lack of real-time, 

In [70]:
def write_text_to_file(text, file_name):
    with open(file_name, "w") as f:
        f.write(text)
        
write_text_to_file(formatted_text, "file_27.txt")

In [46]:
# from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader, UnstructuredPDFLoader
# from typing import Listpy

In [35]:
loader = PyPDFLoader("proposals/file_27.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'proposals/file_27.pdf', 'page': 0}, page_content='TEAM 70:\nDATASET 5: \nShelf Behaviors CCTV Data'),
 Document(metadata={'source': 'proposals/file_27.pdf', 'page': 1}, page_content='TEAM\nMEMBERS\nLe Thuong\nMember\nTrung Dung\nTeam Leader\nThe Vinh\nMember\nThanh Hao\nMember\nHai Yen\nMember'),
 Document(metadata={'source': 'proposals/file_27.pdf', 'page': 2}, page_content='Agenda\nIntroduction \nProblem Statement\nSolution Overview\nMethodologies\nCore Functionality\nPerformance Metrics\nTimeline and Roadmap\nUser Interface (UI) or Interaction\nLimitations and Future Enhancements\nConclusion\n1.\n2.\n3.\n4.\n5.\n6.\n7.\n8.\n9.\n10.'),
 Document(metadata={'source': 'proposals/file_27.pdf', 'page': 3}, page_content='Proposal: Develop an AI solution utilizing video surveillance data\n for in-depth analysis of customer behavior.\n1.INTRO:\nIntroduction to the importance of understanding \ncustomer behavior in retail environments.'),
 Document(metadata={'so

In [36]:
text = " ".join(list(map(lambda page: page.page_content, pages)))
text

"TEAM 70:\nDATASET 5: \nShelf Behaviors CCTV Data TEAM\nMEMBERS\nLe Thuong\nMember\nTrung Dung\nTeam Leader\nThe Vinh\nMember\nThanh Hao\nMember\nHai Yen\nMember Agenda\nIntroduction \nProblem Statement\nSolution Overview\nMethodologies\nCore Functionality\nPerformance Metrics\nTimeline and Roadmap\nUser Interface (UI) or Interaction\nLimitations and Future Enhancements\nConclusion\n1.\n2.\n3.\n4.\n5.\n6.\n7.\n8.\n9.\n10. Proposal: Develop an AI solution utilizing video surveillance data\n for in-depth analysis of customer behavior.\n1.INTRO:\nIntroduction to the importance of understanding \ncustomer behavior in retail environments. Challenges and inefficiencies faced by retail businesses.\nSuboptimal store layout due to a lack of understanding of customer navigation.\nIneffective marketing strategies without detailed customer insights.\nDecreased customer satisfaction due to a lack of understanding of needs and\npreferences.\n2.  PROBLEM STATEMENT\nDEFINING THE PROBLEM\nCore issue ad

In [72]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

def upload_file_to_azure(file_path, container_name=os.getenv("CONTAINER_NAME"), connection_string=os.getenv("BLOB_CONNECTION_STRING")):
    # Create a BlobServiceClient using the connection string
    blob_service_client = BlobServiceClient.from_connection_string(
        connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client(container_name)

    # Get the blob client
    blob_name = os.path.basename(file_path)
    blob_client = container_client.get_blob_client(blob_name)

    # Upload the file
    with open(file_path, "rb") as data:
        blob_client.upload_blob(data)

    print(
        f"File {file_path} uploaded to Azure Blob Storage in container {container_name}.")

In [ ]:
upload_file_to_azure('file_27.txt')

File file_27..txt uploaded to Azure Blob Storage in container extracted-data.
